In [16]:
import os

import requests
import bs4

from fpdf import FPDF
from PyPDF2 import PdfFileMerger, PdfFileReader
from PIL import Image

In [37]:
download_folder = "xkcd"

url = "https://xkcd.com"
# url = "http://www.google.de"
os.makedirs(download_folder, exist_ok=True)

for i in range(20):
    res =  requests.get(url)
    res.raise_for_status()
    
    soup = bs4.BeautifulSoup(res.text, "html.parser")
    
    comic_element = soup.select("#comic img")
    if comic_element == []:
        print("Could not find")
    else:
        # Bild heruntergeladen und gespeichert
        comic_url = 'https:' + comic_element[0].get('src')
        try:
            res = requests.get(comic_url)
        except Exception:
            print("could not download comic")
        with open(os.path.join(download_folder, os.path.basename(comic_url)), "wb") as file:
            for chunk in res.iter_content(100000):
                file.write(chunk)
    prev_link = soup.select('a[rel="prev"]')[0]
    url = f"https://xkcd.com{prev_link.get('href')}"
    

In [38]:
max_width = 500
max_heigh = 650

writer = PdfFileMerger()

print("start seite")
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', '', 50)
pdf.write(50, f"\nxkcd comics")
pdf.output(os.path.join(download_folder, "Start_page.pdf"))

writer.append(PdfFileReader(open(os.path.join(download_folder, "Start_page.pdf"), "rb")))

for index, image in enumerate(os.listdir(download_folder)):
    if image.endswith("png"):
        image2 = Image.open(os.path.join(download_folder, image))
        x,y,w,h = image2.getbbox()
        if w > max_width or h > max_heigh:
            print("resize")
            image2.thumbnail((max_width, max_heigh), Image.ANTIALIAS)
            image2.save(os.path.join(download_folder, image))
            image2.close()

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font('Arial', '', 30)
        pdf.write(10, f'{index+1} - {image[:-4].replace("_", " ")}')
        pdf.image(os.path.join(download_folder, image), 10,50)
        pdf_file_name = f'{image[:-4]}.png'
        pdf.output(os.path.join(download_folder, pdf_file_name))

        writer.append(PdfFileReader(open(os.path.join(download_folder, pdf_file_name), "rb")))

writer.write("comics.pdf")
        
        
        
        
        
        
        
        
        
        






start seite
resize
resize
resize
resize
resize
resize
resize
